In [20]:
import os
import numpy as np
import ROOT
from ROOT import TFile, TCanvas, TPad, TLegend, TMath, TPaveText, 
c = ROOT.TCanvas()

directories = ("p_no_recoils", "p_recoils", "p_vacuum")

root_files_dict = {}

for directory in directories:  
    sim = sorted(f for f in os.listdir(directory) if f.endswith(".root"))  # Only ROOT files  
    root_files = [ROOT.TFile.Open(os.path.join(directory, f)) for f in sim[:4]]  # Open only ROOT files  
    root_files_dict[directory] = [f for f in root_files if f and not f.IsZombie()]  # Remove failed files  

histograms = ["hPtTrack", "hPtJet_R04", "hPtJetDPhiSignal_R04", "hNEvent", "hNtrig", "hPtGammaRatioPtJetDPhi_R04","hPtGammaRatioPtJetDPhi_R02","hPtGammaRatioPtJetDPhi_R03","hPtGammaRatioPtJetDPhi_R05"]
data = {hist: {key: [] for key in directories} for hist in histograms}

for key, files in root_files_dict.items():
    for file in files:
        for hist in histograms:
            hist_obj = file.Get(hist)
            if hist_obj:
                data[hist][key].append(hist_obj.Clone())  # Clone ensures independent objects


print(data["hPtJetDPhiSignal_R04"]["p_vacuum"][0])



Name: hPtJetDPhiSignal_R04 Title: jet pt vs dphi R=0.4 NbinsX: 150


In [21]:

print(data["hNEvent"]["p_vacuum"][0])

hEvents_vac = []
hEvents_nr = []
hEvents_r = []

ranges = range(len(data["hPtTrack"]["p_vacuum"]))
for evt in ranges:
    evt_vac_hist = data["hNEvent"]["p_vacuum"][evt]
    evt_nr_hist = data["hNEvent"]["p_no_recoils"][evt]
    evt_r_hist = data["hNEvent"]["p_recoils"][evt]

    hEvents_vac.append(evt_vac_hist)
    hEvents_r.append(evt_r_hist)
    hEvents_nr.append(evt_nr_hist)

    

vac_Entries = []
r_Entries = []
nr_Entries = []


for i in range(len(hEvents_vac)):
    ve = hEvents_vac[i].GetEntries() 
    nre = hEvents_nr[i].GetEntries()  
    re = hEvents_r[i].GetEntries()  

    # Store the values in the respective lists
    vac_Entries.append(ve)
    r_Entries.append(re)
    nr_Entries.append(nre)   
    
print(r_Entries[0])



print(data["hNtrig"]["p_vacuum"][0])

tEvents_vac = []
tEvents_nr = []
tEvents_r = []

ranges = range(len(data["hNtrig"]["p_vacuum"]))
for evt in ranges:
    t_vac_hist = data["hNtrig"]["p_vacuum"][evt]
    t_nr_hist = data["hNtrig"]["p_no_recoils"][evt]
    t_r_hist = data["hNtrig"]["p_recoils"][evt]

    tEvents_vac.append(t_vac_hist)
    tEvents_r.append(t_r_hist)
    tEvents_nr.append(t_nr_hist)

    

vac_trig = []
r_trig = []
nr_trig = []


for i in range(len(hEvents_vac)):
    vt = tEvents_vac[i].GetBinContent(2)  
    nrt = tEvents_nr[i].GetBinContent(2)    
    rt = tEvents_r[i].GetBinContent(2)    

    # Store the values in the respective lists
    vac_trig.append(vt)
    r_trig.append(rt)
    nr_trig.append(nrt)   
    
print(r_trig[0])
print(len(r_trig))

%jsroot on


abc = ROOT.TCanvas("abc", "Canvas", 1200, 1600)
abc.Divide(1, 2)

# Scale the histograms directly
for i in range(len(data["hPtJet_R04"]["p_vacuum"])):
    data["hPtTrack"]["p_vacuum"][i].Scale(1/vac_Entries[i])
    data["hPtJet_R04"]["p_vacuum"][i].Scale(1/vac_Entries[i])
#    data["hPtJetDPhiSignal_R04"]["p_vacuum"][i].Scale(1/vac_trig[i]) 

for i in range(len(data["hPtJet_R04"]["p_no_recoils"])):
    data["hPtTrack"]["p_no_recoils"][i].Scale(1/nr_Entries[i])
    data["hPtJet_R04"]["p_no_recoils"][i].Scale(1/nr_Entries[i])
    data["hPtTrack"]["p_no_recoils"][i].Divide(data["hPtTrack"]["p_vacuum"][i])
    data["hPtJet_R04"]["p_no_recoils"][i].Divide(data["hPtJet_R04"]["p_vacuum"][i])

for i in range(len(data["hPtJet_R04"]["p_recoils"])):
    data["hPtTrack"]["p_recoils"][i].Scale(1/r_Entries[i])
    data["hPtJet_R04"]["p_recoils"][i].Scale(1/r_Entries[i])
    data["hPtTrack"]["p_recoils"][i].Divide(data["hPtTrack"]["p_vacuum"][i])
    data["hPtJet_R04"]["p_recoils"][i].Divide(data["hPtJet_R04"]["p_vacuum"][i])

 
                 


Name: hNEvent Title: number of events NbinsX: 1
2033111.0
Name: hNtrig Title: number of triggers NbinsX: 2
0.706308126449585
4


Warning in <TCanvas::Constructor>: Deleting canvas with same name: abc


In [22]:
def axis_up(x1, x2, c):
    y = c.GetUymax()
    axis_top = ROOT.TGaxis(x1, y, x2, y, x1, x2, 510, "+")
    axis_top.SetLabelFont(40)  # plain font (not bold)
    axis_top.SetTitleFont(40)
    return axis_top
    
def axis_right(y1, y2, xr):
    
    axis_right = ROOT.TGaxis(xr, y1, xr, y2, y1, y2, 510, "+")
    axis_right.SetLabelFont(42)
    axis_right.SetTitleFont(42)
    return axis_right

In [26]:
#~~~~
bin_fin_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].GetZaxis()
first_bin_z = bin_fin_z.FindBin(1.4+0.1)
last_bin_z = bin_fin_z.FindBin(3.14159-0.1)

bin_fin_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].GetYaxis()
first_bin_y = bin_fin_y.FindBin(10+0.1)
last_bin_y = bin_fin_y.FindBin(100-0.1)

trig_bin_nr = data["hNtrig"]["p_no_recoils"][3].GetBinContent(2)

nr_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].ProjectionX("interval_nr", iymin = first_bin_y, iymax = last_bin_y, izmin = first_bin_z, izmax = last_bin_z)
nr_interval.SetTitle("no p_recoils Jet pT Photon E ratio")
#nr_interval.Rebin(3)
nr_interval.Scale(1.0 / trig_bin_nr,"width")
#scale so that all are the same integral for now
#nr_interval.Scale(3.0 / nr_interval.Integral())
#nr_interval.Fit(f_gauss, "R")
nr_interval.SetLineColor(ROOT.kPink-3)

#~~~~~~~~~~~~~~~~~~~
bin_fin_vac_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].GetZaxis()
first_bin_vac_z = bin_fin_vac_z.FindBin(1.4+0.1)
last_bin_vac_z = bin_fin_vac_z.FindBin(3.14159-0.1)

bin_fin_vac_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].GetYaxis()
first_bin_vac_y = bin_fin_y.FindBin(10+0.1)
last_bin_vac_y = bin_fin_y.FindBin(100-0.1)

trig_bin_vac = data["hNtrig"]["p_vacuum"][3].GetBinContent(2)

vac_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].ProjectionX("interval_vac", iymin = first_bin_vac_y, iymax = last_bin_vac_y, izmin = first_bin_vac_z, izmax = last_bin_vac_z)
vac_interval.SetTitle("p_vacuum Jet pT Photon E ratio")
#nr_interval.Rebin(3)
vac_interval.Scale(1.0 / trig_bin_vac,"width")
#scale so that all are the same integral for now
#nr_interval.Scale(1.0 / nr_interval.Integral())
#nr_interval.Fit(f_gauss, "R")
vac_interval.SetLineColor(ROOT.kBlue-3)

#~~~~~~~~~~~~~~~

bin_fin_r_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].GetZaxis()
first_bin_r_z = bin_fin_z.FindBin(1.4+0.1)
last_bin_r_z = bin_fin_z.FindBin(3.14159-0.1)

bin_fin_r_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].GetYaxis()
first_bin_r_y = bin_fin_y.FindBin(10+0.1)
last_bin_r_y = bin_fin_y.FindBin(100-0.1)

trig_bin_r = data["hNtrig"]["p_recoils"][3].GetBinContent(2)

r_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].ProjectionX("interval_r", iymin = first_bin_r_y, iymax = last_bin_r_y, izmin = first_bin_r_z, izmax = last_bin_r_z)
r_interval.SetTitle("p_recoils Jet pT Photon E ratio")
#nr_interval.Rebin(3)
r_interval.Scale(1.0 / trig_bin_r,"width")
#scale so that all are the same integral for now
#nr_interval.Scale(1.0 / nr_interval.Integral())
#nr_interval.Fit(f_gauss, "R")
r_interval.SetLineColor(ROOT.kViolet-3)

#~~~~~~~~~~~~~~
ROOT.gStyle.SetOptStat(0)

abcdefg = ROOT.TCanvas("abcdefg", "Canvas", 600, 400)
#ROOT.gPad.SetLogy(1)
nr_interval.SetStats(False)
nr_interval.GetXaxis().SetRangeUser(0,6)
nr_interval.GetYaxis().SetRangeUser(0.0,5.0)
#nr_interval.SetTitle("hadron+jet #Delta#phi distribution, jet pT = 10-200 GeV")
nr_interval.Draw()
vac_interval.Draw("SAME")
r_interval.Draw("SAME")
#nr_interval.GetYaxis().SetTitle("number of recoiling jets (arb. normalisation)")


legendP = TLegend(0.2,0.55,0.4,0.85)
legendP.SetBorderSize(0)
legendP.SetFillColor(0)

legtext1 =  "No p_recoils 30-50"
legtext2 = "p_recoils 30-50"
legtext3 = "p_vacuum 30-50"
#legtext4 = formatted_string = f"#splitline{{hadron pT = 50-70 GeV}}{{#sigma = {fit_func4.GetParameter(1):.2f} ± {fit_func4.GetParError(1):.2f}}}"
legendP.AddEntry(nr_interval,legtext1, "l")
legendP.AddEntry(r_interval, legtext2, "l")
legendP.AddEntry(vac_interval, legtext3, "l")
#legendP.AddEntry(nr_interval4, legtext4, "l")
legendP.Draw()

abcdefg.Draw()
abcdefg.SaveAs("Ratio_NR.png")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: abcdefg
Info in <TCanvas::Print>: png file Ratio_NR.png has been created


In [31]:
#~~~~
bin_fin_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][2].GetZaxis()
first_bin_z = bin_fin_z.FindBin(1.5+0.1)
last_bin_z = bin_fin_z.FindBin(3.14159-0.1)

bin_fin_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][2].GetYaxis()
first_bin_y = bin_fin_y.FindBin(9+0.1)
last_bin_y = bin_fin_y.FindBin(200-0.1)

nr_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][2].ProjectionX("interval_nr", iymin = first_bin_y, iymax = last_bin_y, izmin = first_bin_z, izmax = last_bin_z)

nr_interval.Scale(1.0 / nr_interval.Integral())
#nr_interval.Scale(1.0 / nr_trig[2])#,"width")
nr_interval.SetTitle("  ")

nr_interval.SetLineColor(ROOT.kPink-3)

#~~~~~~~~~~~~~~~~~~~
bin_fin_vac_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][2].GetZaxis()
first_bin_vac_z = bin_fin_vac_z.FindBin(1.5+0.1)
last_bin_vac_z = bin_fin_vac_z.FindBin(3.14159-0.1)

bin_fin_vac_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][2].GetYaxis()
first_bin_vac_y = bin_fin_y.FindBin(9+0.1)
last_bin_vac_y = bin_fin_y.FindBin(200-0.1)

vac_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][2].ProjectionX("interval_vac", iymin = first_bin_vac_y, iymax = last_bin_vac_y, izmin = first_bin_vac_z, izmax = last_bin_vac_z)

vac_interval.Scale(1.0 / vac_interval.Integral())
#vac_interval.Scale(1.0 / vac_trig[2])#,"width")

vac_interval.SetLineColor(ROOT.kBlue-3)

#~~~~~~~~~~~~~~~

bin_fin_r_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][2].GetZaxis()
first_bin_r_z = bin_fin_z.FindBin(1.5+0.1)
last_bin_r_z = bin_fin_z.FindBin(3.14159-0.1)

bin_fin_r_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][2].GetYaxis()
first_bin_r_y = bin_fin_y.FindBin(9+0.1)
last_bin_r_y = bin_fin_y.FindBin(200-0.1)


r_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][2].ProjectionX("interval_r", iymin = first_bin_r_y, iymax = last_bin_r_y, izmin = first_bin_r_z, izmax = last_bin_r_z)

#r_interval.Scale(1.0 / r_trig[2])
r_interval.Scale(1.0 / r_interval.Integral())

r_interval.SetLineColor(ROOT.kViolet-3)

#~~~~~~~~~~~~~~
ROOT.gStyle.SetOptStat(0)

abcdefg = ROOT.TCanvas("abcdefg", "Canvas", 600, 400)
#ROOT.gPad.SetLogy(1)
nr_interval.SetStats(False)
nr_interval.GetXaxis().SetRangeUser(0,2)
nr_interval.GetYaxis().SetRangeUser(0.0,0.3)
#nr_interval.SetTitle("photon+jet #Delta#phi distribution, jet pT = 10-200 GeV")
nr_interval.Draw()
vac_interval.Draw("SAME")
r_interval.Draw("SAME")
#nr_interval.GetYaxis().SetTitle("number of recoiling jets (arb. normalisation)")


legendP = TLegend(0.6,0.7,0.85,0.85)
legendP.SetBorderSize(0)
legendP.SetFillColor(0)
legtext1 =  "no recoils"
legtext2 = "recoils"
legtext3 = "vacuum"
#legtext4 = formatted_string = f"#splitline{{photon pT = 50-70 GeV}}{{#sigma = {fit_func4.GetParameter(1):.2f} ± {fit_func4.GetParError(1):.2f}}}"
legendP.AddEntry(nr_interval,legtext1, "l")
legendP.AddEntry(r_interval, legtext2, "l")
legendP.AddEntry(vac_interval, legtext3, "l")
#legendP.AddEntry(nr_interval4, legtext4, "l")
legendP.Draw()

abcdefg.Draw()
abcdefg.SaveAs("Ratio_NR.png")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: abcdefg


In [124]:

abcdef = ROOT.TCanvas("abcdef", "Canvas", 1200, 800)


bin_fin_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].GetZaxis()
first_bin_z = bin_fin_z.FindBin(1.4+0.1)
last_bin_z = bin_fin_z.FindBin(3.14159-0.1)
bin_fin_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].GetYaxis()
first_bin_y = bin_fin_y.FindBin(10+0.1)
last_bin_y = bin_fin_y.FindBin(200-0.1)

nr_interval1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].ProjectionX("interval_nr", iymin = first_bin_y, iymax = last_bin_y, izmin = first_bin_z, izmax = last_bin_z)
nr_interval1.GetYaxis().SetTitle("Arb. Normalisation")
nr_interval1.SetTitle(" ")
#nr_interval.Rebin(3)
#nr_interval1.Scale(1 / nr_trig[3], "width")
nr_interval.Scale(1.0 / nr_interval.Integral())

nr_interval1.SetLineColor(ROOT.kPink-3)
#~~~~~~~~~~~~~~~~~~~
bin_fin_z1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].GetZaxis()
first_bin_z1 = bin_fin_z1.FindBin(1.4+0.1)
last_bin_z1 = bin_fin_z1.FindBin(3.14159-0.1)
bin_fin_y1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].GetYaxis()
first_bin_y1 = bin_fin_y1.FindBin(10+0.1)
last_bin_y1 = bin_fin_y1.FindBin(200-0.1)

vac_interval1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].ProjectionX("interval_vac", iymin = first_bin_y1, iymax = last_bin_y1, izmin = first_bin_z1, izmax = last_bin_z1)
vac_interval1.SetTitle("p_vacuum Jet pT Photon E ratio")
#vac_interval1.Scale(1/ vac_trig[3], "width")
vac_interval1.Scale(1.0 / vac_interval.Integral())
vac_interval1.SetLineColor(ROOT.kBlue-3)

#~~~~~~~~~~~~~~~

bin_fin_z2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].GetZaxis()
first_bin_z2 = bin_fin_z2.FindBin(1.4+0.1)
last_bin_z2 = bin_fin_z2.FindBin(3.14159-0.1)

bin_fin_y2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].GetYaxis()
first_bin_y2 = bin_fin_y2.FindBin(10+0.1)
last_bin_y2 = bin_fin_y2.FindBin(200-0.1)
r_interval1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].ProjectionX("interval_r", iymin = first_bin_y2, iymax = last_bin_y2, izmin = first_bin_z2, izmax = last_bin_z2)
r_interval1.SetTitle("p_recoils Jet pT Photon E ratio")
#r_interval1.Scale(1 / r_trig[3], "width")
r_interval.Scale(1.0 / r_interval.Integral())
r_interval1.SetLineColor(ROOT.kViolet-3)

#~~~~~~~~~~~~~~
ROOT.gStyle.SetOptStat(0)

x1 = 0
x2 = 2
y1 = 0
y2 = 0.3


nr_interval1.SetStats(False)
nr_interval1.GetXaxis().SetRangeUser(x1,x2)
nr_interval1.GetYaxis().SetRangeUser(y1,y2)

nr_interval1.Draw("E1")
vac_interval1.Draw("E1SAME")
r_interval1.Draw("E1SAME")

#legendP = TPaveText(1.43,0.08,1.8,0.13, option = "br")
legendP = TPaveText(1.43,0.05,1.8,0.07, option = "br")
legendP.SetBorderSize(0)
legendP.SetFillColor(0)
legtext = "50 - 70 GeV/c"
legendP.AddText(legtext)

legendP1 = TLegend(0.6,0.7,0.85,0.85)
legendP1.SetBorderSize(0)
legendP1.SetFillColor(0)
legtext1 =  "No Recoils"
legtext2 = "Recoils"
legtext3 = "Vacuum"
legendP1.AddEntry(nr_interval1, legtext1, "l")
legendP1.AddEntry(r_interval1, legtext2, "l")
legendP1.AddEntry(vac_interval1, legtext3, "l")

legendP1.Draw()

abcdef.Draw()
legendP.Draw()
abcdef.SaveAs("Photon_Graphs/ptoverE5070.png")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: abcdef
Info in <TCanvas::Print>: png file Photon_Graphs/ptoverE5070.png has been created


In [11]:
abcdefg = ROOT.TCanvas("abcdefg", "Canvas", 1600, 2800)
abcdefg.Divide(2, 2)
abcdefg.cd(1)

bin_fin_z1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][0].GetZaxis()
first_bin_z1 = bin_fin_z1.FindBin(1.4+0.1)
last_bin_z1 = bin_fin_z1.FindBin(3.14259-0.1)
bin_fin_y1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][0].GetYaxis()
first_bin_y1 = bin_fin_y1.FindBin(10+0.1)
last_bin_y1 = bin_fin_y1.FindBin(100-0.1)

trig_bin_nr1 = data["hNtrig"]["p_no_recoils"][0].GetBinContent(2)

nr_interval1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][0].ProjectionX("interval_nr", iymin = first_bin_y1, iymax = last_bin_y1, izmin = first_bin_z1, izmax = last_bin_z1)
nr_interval1.GetYaxis().SetTitle("Arb. Normalisation")
nr_interval1.SetTitle("no recoils Jet pT Photon E ratio")
#nr_interval.Rebin(3)
nr_interval1.Scale(1.0 / r_trig[0])#,"width")
#scale so that all are the same integral for now
#nr_interval.Scale(2.0 / nr_interval.Integral())
#nr_interval.Fit(f_gauss, "R")
nr_interval1.SetLineColor(ROOT.kPink-3)
#~~~~~~~~~~~~~~~~~~~
bin_fin_vac_z1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][0].GetZaxis()
first_bin_vac_z1 = bin_fin_vac_z1.FindBin(1.4+0.1)
last_bin_vac_z1 = bin_fin_vac_z1.FindBin(3.14259-0.1)
bin_fin_vac_y1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][0].GetYaxis()
first_bin_vac_y1 = bin_fin_y1.FindBin(10+0.1)
last_bin_vac_y1 = bin_fin_y1.FindBin(100-0.1)
trig_bin_vac1 = data["hNtrig"]["p_vacuum"][0].GetBinContent(2)
vac_interval1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][0].ProjectionX("interval_vac", iymin = first_bin_vac_y1, iymax = last_bin_vac_y1, izmin = first_bin_vac_z1, izmax = last_bin_vac_z1)
vac_interval1.SetTitle("p_vacuum Jet pT Photon E ratio")
vac_interval1.Scale(1.0 / vac_trig[0])#,"width")
vac_interval1.SetLineColor(ROOT.kBlue-3)

#~~~~~~~~~~~~~~~

bin_fin_r_z1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][0].GetZaxis()

first_bin_r_z1 = bin_fin_z1.FindBin(1.4+0.1)
last_bin_r_z1 = bin_fin_z1.FindBin(3.14259-0.1)

bin_fin_r_y1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][0].GetYaxis()
first_bin_r_y1 = bin_fin_y1.FindBin(10+0.1)
last_bin_r_y1 = bin_fin_y1.FindBin(100-0.1)
r_interval1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][0].ProjectionX("interval_r", iymin = first_bin_r_y1, iymax = last_bin_r_y1, izmin = first_bin_r_z1, izmax = last_bin_r_z1)
r_interval1.SetTitle("p_recoils Jet pT Photon E ratio")
r_interval1.Scale(1.0 / r_trig[0])#,"width")
r_interval1.SetLineColor(ROOT.kViolet-3)

#~~~~~~~~~~~~~~
ROOT.gStyle.SetOptStat(0)

nr_interval1.SetStats(False)
nr_interval1.GetXaxis().SetRangeUser(x1,x2)
nr_interval1.GetYaxis().SetRangeUser(y1,y2)

nr_interval1.Draw()
vac_interval1.Draw("SAME")
r_interval1.Draw("SAME")

legendP1 = TLegend(0.6,0.7,0.85,0.85)
legendP1.SetBorderSize(0)
legendP1.SetFillColor(0)

legendP1.AddEntry(nr_interval1, legtext1, "l")
legendP1.AddEntry(r_interval1, legtext2, "l")
legendP1.AddEntry(vac_interval1, legtext3, "l")

legendP1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: abcdefg


In [12]:

abcdefg.cd(2)

bin_fin_z2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][1].GetZaxis()
first_bin_z2 = bin_fin_z2.FindBin(1.4+0.1)
last_bin_z2 = bin_fin_z2.FindBin(3.14259-0.1)
bin_fin_y2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][1].GetYaxis()
first_bin_y2 = bin_fin_y2.FindBin(10+0.1)
last_bin_y2 = bin_fin_y2.FindBin(100-0.1)

trig_bin_nr2 = data["hNtrig"]["p_no_recoils"][1].GetBinContent(2)

nr_interval2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][1].ProjectionX("interval_nr", iymin = first_bin_y2, iymax = last_bin_y2, izmin = first_bin_z2, izmax = last_bin_z2)
nr_interval2.GetYaxis().SetTitle("Arb. Normalisation")
nr_interval2.SetTitle("no p_recoils Jet pT Photon E ratio")
#nr_interval.Rebin(3)
nr_interval2.Scale(1.0 / r_trig[1])#,"width")
#scale so that all are the same integral for now
#nr_interval.Scale(2.0 / nr_interval.Integral())
#nr_interval.Fit(f_gauss, "R")
nr_interval2.SetLineColor(ROOT.kPink-3)
#~~~~~~~~~~~~~~~~~~~
bin_fin_vac_z2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][1].GetZaxis()
first_bin_vac_z2 = bin_fin_vac_z2.FindBin(1.4+0.1)
last_bin_vac_z2 = bin_fin_vac_z2.FindBin(3.14259-0.1)
bin_fin_vac_y2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][1].GetYaxis()
first_bin_vac_y2 = bin_fin_y2.FindBin(10+0.1)
last_bin_vac_y2 = bin_fin_y2.FindBin(100-0.1)
trig_bin_vac2 = data["hNtrig"]["p_vacuum"][1].GetBinContent(2)
vac_interval2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][1].ProjectionX("interval_vac", iymin = first_bin_vac_y2, iymax = last_bin_vac_y2, izmin = first_bin_vac_z2, izmax = last_bin_vac_z2)
vac_interval2.SetTitle("p_vacuum Jet pT Photon E ratio")
vac_interval2.Scale(1.0 / vac_trig[1])#,"width")
vac_interval2.SetLineColor(ROOT.kBlue-3)

#~~~~~~~~~~~~~~~

bin_fin_r_z2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][1].GetZaxis()
first_bin_r_z2 = bin_fin_z2.FindBin(1.4+0.1)
last_bin_r_z2 = bin_fin_z2.FindBin(3.14259-0.1)

bin_fin_r_y2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][1].GetYaxis()
first_bin_r_y2 = bin_fin_y2.FindBin(10+0.1)
last_bin_r_y2 = bin_fin_y2.FindBin(100-0.1)
r_interval2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][1].ProjectionX("interval_r", iymin = first_bin_r_y2, iymax = last_bin_r_y2, izmin = first_bin_r_z2, izmax = last_bin_r_z2)
r_interval2.SetTitle("p_recoils Jet pT Photon E ratio")
r_interval2.Scale(1.0 / r_trig[1])#,"width")
r_interval2.SetLineColor(ROOT.kViolet-3)

#~~~~~~~~~~~~~~
ROOT.gStyle.SetOptStat(0)

nr_interval2.SetStats(False)
nr_interval2.GetXaxis().SetRangeUser(x1,x2)
nr_interval2.GetYaxis().SetRangeUser(y1,y2)

nr_interval2.Draw()
vac_interval2.Draw("SAME")
r_interval2.Draw("SAME")

legendP2 = TLegend(0.6,0.7,0.85,0.85)
legendP2.SetBorderSize(0)
legendP2.SetFillColor(0)

legendP2.AddEntry(nr_interval2, legtext1, "l")
legendP2.AddEntry(r_interval2, legtext2, "l")
legendP2.AddEntry(vac_interval2, legtext3, "l")

legendP2.Draw()

In [13]:
abcdefg.cd(3)

bin_fin_z3 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][2].GetZaxis()
first_bin_z3 = bin_fin_z3.FindBin(1.4+0.1)
last_bin_z3 = bin_fin_z3.FindBin(3.14159-0.1)
bin_fin_y3 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][2].GetYaxis()
first_bin_y3 = bin_fin_y3.FindBin(10+0.1)
last_bin_y3 = bin_fin_y3.FindBin(100-0.1)

nr_interval3 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][2].ProjectionX("interval_nr", iymin = first_bin_y3, iymax = last_bin_y3, izmin = first_bin_z3, izmax = last_bin_z3)
nr_interval3.GetYaxis().SetTitle("Arb. Normalisation")
nr_interval3.SetTitle("no p_recoils Jet pT Photon E ratio")
#nr_interval.Rebin(3)
nr_interval3.Scale(1.0 / r_trig[2])#,"width")
#scale so that all are the same integral for now
#nr_interval.Scale(3.0 / nr_interval.Integral())
#nr_interval.Fit(f_gauss, "R")
nr_interval3.SetLineColor(ROOT.kPink-3)
#~~~~~~~~~~~~~~~~~~~

bin_fin_vac_z3 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][2].GetZaxis()
first_bin_vac_z3 = bin_fin_vac_z3.FindBin(1.4+0.1)
last_bin_vac_z3 = bin_fin_vac_z3.FindBin(3.14159-0.1)
bin_fin_vac_y3 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][2].GetYaxis()
first_bin_vac_y3 = bin_fin_y3.FindBin(10+0.1)
last_bin_vac_y3 = bin_fin_y3.FindBin(100-0.1)

vac_interval3 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][2].ProjectionX("interval_vac", iymin = first_bin_vac_y3, iymax = last_bin_vac_y3, izmin = first_bin_vac_z3, izmax = last_bin_vac_z3)
vac_interval3.SetTitle("p_vacuum Jet pT Photon E ratio")
vac_interval3.Scale(1.0 / vac_trig[2])#,"width")
vac_interval3.SetLineColor(ROOT.kBlue-3)

#~~~~~~~~~~~~~~~

bin_fin_r_z3 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][2].GetZaxis()
first_bin_r_z3 = bin_fin_z3.FindBin(1.4+0.1)
last_bin_r_z3 = bin_fin_z3.FindBin(3.14159-0.1)

bin_fin_r_y3 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][2].GetYaxis()
first_bin_r_y3 = bin_fin_y3.FindBin(10+0.1)
last_bin_r_y3 = bin_fin_y3.FindBin(100-0.1)
r_interval3 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][2].ProjectionX("interval_r", iymin = first_bin_r_y3, iymax = last_bin_r_y3, izmin = first_bin_r_z3, izmax = last_bin_r_z3)
r_interval3.SetTitle("p_recoils Jet pT Photon E ratio")
r_interval3.Scale(1.0 / r_trig[2])#,"width")
r_interval3.SetLineColor(ROOT.kViolet-3)

#~~~~~~~~~~~~~~
ROOT.gStyle.SetOptStat(0)

nr_interval3.SetStats(False)
nr_interval3.GetXaxis().SetRangeUser(x1,x2)
nr_interval3.GetYaxis().SetRangeUser(y1,y2)

nr_interval3.Draw()
vac_interval3.Draw("SAME")
r_interval3.Draw("SAME")


legendP3 = TLegend(0.6,0.7,0.85,0.85)
legendP3.SetBorderSize(0)
legendP3.SetFillColor(0)

legendP3.AddEntry(nr_interval3, legtext1, "l")
legendP3.AddEntry(r_interval3, legtext3, "l")
legendP3.AddEntry(vac_interval3, legtext3, "l")

legendP3.Draw()

In [14]:
abcdefg.cd(4)

bin_fin_z4 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].GetZaxis()
first_bin_z4 = bin_fin_z4.FindBin(1.4+0.1)
last_bin_z4 = bin_fin_z4.FindBin(3.14159-0.1)
bin_fin_y4 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].GetYaxis()
first_bin_y4 = bin_fin_y4.FindBin(10+0.1)
last_bin_y4 = bin_fin_y4.FindBin(100-0.1)

trig_bin_nr4 = data["hNtrig"]["p_no_recoils"][3].GetBinContent(4)

nr_interval4 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].ProjectionX("interval_nr", iymin = first_bin_y4, iymax = last_bin_y4, izmin = first_bin_z4, izmax = last_bin_z4)
nr_interval4.GetYaxis().SetTitle("Arb. Normalisation")
nr_interval4.SetTitle("no p_recoils Jet pT Photon E ratio")
#nr_interval.Rebin(3)
nr_interval4.Scale(1.0 / r_trig[3])#,"width")
#scale so that all are the same integral for now
#nr_interval.Scale(4.0 / nr_interval.Integral())
#nr_interval.Fit(f_gauss, "R")
nr_interval4.SetLineColor(ROOT.kPink-3)
#~~~~~~~~~~~~~~~~~~~
bin_fin_vac_z4 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].GetZaxis()
first_bin_vac_z4 = bin_fin_vac_z4.FindBin(1.4+0.1)
last_bin_vac_z4 = bin_fin_vac_z4.FindBin(3.14159-0.1)
bin_fin_vac_y4 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].GetYaxis()
first_bin_vac_y4 = bin_fin_y4.FindBin(10+0.1)
last_bin_vac_y4 = bin_fin_y4.FindBin(100-0.1)
trig_bin_vac4 = data["hNtrig"]["p_vacuum"][3].GetBinContent(4)
vac_interval4 = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].ProjectionX("interval_vac", iymin = first_bin_vac_y4, iymax = last_bin_vac_y4, izmin = first_bin_vac_z4, izmax = last_bin_vac_z4)
vac_interval4.SetTitle("p_vacuum Jet pT Photon E ratio")
vac_interval4.Scale(1.0 / vac_trig[3])#,"width")
vac_interval4.SetLineColor(ROOT.kBlue-3)

#~~~~~~~~~~~~~~~

bin_fin_r_z4 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].GetZaxis()
first_bin_r_z4 = bin_fin_z4.FindBin(1.4+0.1)
last_bin_r_z4 = bin_fin_z4.FindBin(3.14159-0.1)

bin_fin_r_y4 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].GetYaxis()
first_bin_r_y4 = bin_fin_y4.FindBin(10+0.1)
last_bin_r_y4 = bin_fin_y4.FindBin(100-0.1)
r_interval4 = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].ProjectionX("interval_r", iymin = first_bin_r_y4, iymax = last_bin_r_y4, izmin = first_bin_r_z4, izmax = last_bin_r_z4)
r_interval4.SetTitle("p_recoils Jet pT Photon E ratio")
r_interval4.Scale(1.0 / r_trig[3])#,"width")
r_interval4.SetLineColor(ROOT.kViolet-3)

#~~~~~~~~~~~~~~
ROOT.gStyle.SetOptStat(0)

nr_interval4.SetStats(False)
nr_interval4.GetXaxis().SetRangeUser(x1,x2)
nr_interval4.GetYaxis().SetRangeUser(y1,y2)

nr_interval4.Draw()
vac_interval4.Draw("SAME")
r_interval4.Draw("SAME")


legendP4 = TLegend(0.6,0.7,0.85,0.85)
legendP4.SetBorderSize(0)
legendP4.SetFillColor(0)

legendP4.AddEntry(nr_interval4, legtext1, "l")
legendP4.AddEntry(r_interval4, legtext2, "l")
legendP4.AddEntry(vac_interval4, legtext3, "l")

legendP4.Draw()

In [15]:
abcdefg.Draw()

In [198]:
#~~~~
bin_fin_z1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][0].GetZaxis()
first_bin_z1 = bin_fin_z1.FindBin(1.4+0.1)
last_bin_z1 = bin_fin_z1.FindBin(3.14159-0.1)

bin_fin_y1 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][0].GetYaxis()
first_bin_y1 = bin_fin_y1.FindBin(10+0.1)
last_bin_y1 = bin_fin_y1.FindBin(100-0.1)


bin_fin_z2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][1].GetZaxis()
first_bin_z2 = bin_fin_z2.FindBin(1.4+0.1)
last_bin_z2 = bin_fin_z2.FindBin(3.14159-0.1)

bin_fin_y2 = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][1].GetYaxis()
first_bin_y2 = bin_fin_y2.FindBin(10+0.1)
last_bin_y2 = bin_fin_y2.FindBin(100-0.1)

print(first_bin_y2)


#~~~~~~~~~~~~~~~

# Output containers if needed
histi1 = (data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][0])
histi2 = (data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][1])
histi3 = (data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][2])
histi4 = (data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3])

histis = (histi1,histi2,histi3,histi4)

bin_results = []

for idx, hist in enumerate(histis, start=0):
    zaxis = hist.GetZaxis()
    yaxis = hist.GetYaxis()

    first_bin_z = zaxis.FindBin(1.5)  # 1.4 + 0.1
    last_bin_z = zaxis.FindBin(3.14159)  # 3.14159 - 0.1

    first_bin_y = yaxis.FindBin(10.1)
    last_bin_y = yaxis.FindBin(99.9)

    # Store per histogram result using index
    bin_results.append([first_bin_z, last_bin_z])
    bin_results.append([first_bin_y, last_bin_y])

#bin result - [z1 z2][y1 y2]
print(bin_results)
print(bin_results[0][1])

11
[[23, 48], [11, 100], [23, 48], [11, 100], [23, 48], [11, 100], [23, 48], [11, 100]]
48
